In [1]:
from Parameters import TrainingParameters, EnviromentParameters
from Data import XESDataset
from Models import BaselineLSTM
import tensorflow as tf

%load_ext tensorboard
%tensorboard --logdir logs/gradient_tape --host localhost --port 8088

In [2]:
train_param = TrainingParameters()
xes_dataset = XESDataset(file_path=EnviromentParameters.BPI2020Dataset.file_path,preprocessed_folder_path=EnviromentParameters.BPI2020Dataset.preprocessed_foldr_path,
preprocessed_df_type= EnviromentParameters.BPI2020Dataset.preprocessed_df_type,
include_types = train_param.bpi2012.BPI2012_include_types
)


| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012/OW 


In [3]:
model = BaselineLSTM(
    xes_dataset.vocab, 
    embedding_dim= train_param.baselineLSTMModelParameters.embedding_dim,
    lstm_hidden= train_param.baselineLSTMModelParameters.lstm_hidden,
    dropout=train_param.baselineLSTMModelParameters.dropout
)

In [4]:
optim = tf.keras.optimizers.Adam(learning_rate=0.01)

In [5]:
idx_ds = xes_dataset.get_index_ds()
idx_ds = idx_ds.shuffle(20).batch(train_param.batch_size)

In [6]:
import datetime

In [12]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
tf.keras.callbacks.TensorBoard(log_dir=train_log_dir)

In [15]:
loss_all

<tf.Tensor: shape=(58, 44), dtype=float32, numpy=
array([[0.7091251 , 0.06700239, 0.6059741 , ..., 0.02673648, 0.04303451,
        0.04212036],
       [1.0045804 , 0.00209529, 1.0481571 , ..., 0.02315483, 0.00892164,
        0.00741764],
       [0.9060275 , 0.00323511, 0.8061419 , ..., 0.01034363, 0.01599118,
        0.00780521],
       ...,
       [0.9751311 , 0.00745349, 0.52357495, ..., 0.10319766, 0.15615575,
        0.16799471],
       [0.9090756 , 0.00206555, 0.53607124, ..., 0.12926836, 0.03867538,
        0.01703617],
       [0.52035254, 0.02385254, 0.7061545 , ..., 0.08904197, 0.11406049,
        0.07763471]], dtype=float32)>

In [18]:
loss_all.shape

TensorShape([58, 44])

In [26]:
steps = 0
for idxs in idx_ds:
    caseids, padded_data_traces, padded_target_traces =  xes_dataset.collate_fn(idxs)
    with tf.GradientTape() as tape:
        pred, _ = model(padded_data_traces, training=True)
        loss_all = tf.keras.losses.sparse_categorical_crossentropy(y_true=padded_target_traces, y_pred=pred)
        loss_all = loss_all * tf.cast(padded_target_traces != 0, dtype=tf.float32)
        loss = tf.reduce_mean(loss_all)
        pred_value = tf.math.argmax(pred, axis=-1)
        accuracy = tf.math.reduce_mean(tf.cast(tf.boolean_mask(padded_target_traces == pred_value,padded_target_traces != 0), dtype= tf.float32)).numpy()
        with train_summary_writer.as_default():
            tf.summary.scalar('accuracy', accuracy, step=steps)
            tf.summary.scalar('loss', loss.numpy(), step=steps)
        print("Acc: %.4f" % (accuracy))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
    steps += 1


Acc: 0.6620
Acc: 0.6538
Acc: 0.6664
Acc: 0.7617
Acc: 0.7776
Acc: 0.8137
Acc: 0.8157
Acc: 0.8025
Acc: 0.7943
Acc: 0.8055
Acc: 0.8069
Acc: 0.8133
Acc: 0.8179
Acc: 0.8248
Acc: 0.8274
Acc: 0.8531
Acc: 0.8529
Acc: 0.8339
Acc: 0.8410
Acc: 0.8367
Acc: 0.8487
Acc: 0.8554
Acc: 0.8544
Acc: 0.8485
Acc: 0.8560
Acc: 0.8705
Acc: 0.8640
Acc: 0.8414
Acc: 0.8433
Acc: 0.8551
Acc: 0.8524
Acc: 0.8608
Acc: 0.8547
Acc: 0.8606
Acc: 0.8558
Acc: 0.8583
Acc: 0.8573
Acc: 0.8564
Acc: 0.8559
Acc: 0.8483
Acc: 0.8562
Acc: 0.8309
Acc: 0.8438
Acc: 0.8512
Acc: 0.8353
Acc: 0.8655
Acc: 0.8495
Acc: 0.8482
Acc: 0.8563
Acc: 0.8524
Acc: 0.8453
Acc: 0.8417
Acc: 0.8495
Acc: 0.8454
Acc: 0.8461
Acc: 0.8499
Acc: 0.8526
Acc: 0.8492
Acc: 0.8411
Acc: 0.8482
Acc: 0.8510
Acc: 0.8536
Acc: 0.8489
Acc: 0.8441
Acc: 0.8610
Acc: 0.8376
Acc: 0.8356
Acc: 0.8489
Acc: 0.8384
Acc: 0.8359
Acc: 0.8422
Acc: 0.8492
Acc: 0.8345
Acc: 0.8451
Acc: 0.8276
Acc: 0.8198


In [9]:
accuracy

0.5699615

In [10]:
padded_target_traces != 0

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [11]:
acc_m.update_state(padded_target_traces, pred)

NameError: name 'acc_m' is not defined

In [10]:
padded_target_traces

array([[17, 18,  7, ...,  0,  0,  0],
       [11, 12, 10, ...,  0,  0,  0],
       [11, 12, 10, ...,  0,  0,  0],
       ...,
       [11, 12, 10, ...,  0,  0,  0],
       [11, 12, 17, ...,  0,  0,  0],
       [11, 12, 10, ...,  0,  0,  0]], dtype=int32)

In [11]:
pred_value = tf.math.argmax(pred, axis=-1)

0.59887004

In [12]:
acc_m.result().numpy()

0.85876745

In [13]:
tf.math.reduce_mean(tf.cast(padded_target_traces == pred_value, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.85876745>

<tf.Tensor: shape=(58, 47), dtype=float32, numpy=
array([[0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       ...,
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.]], dtype=float32)>